In [186]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import matplotlib.pyplot as plt
import seaborn as sns


class Embeddings(nn.Module):
    """嵌入层"""

    def __init__(self, d_model, vocab):
        super().__init__()
        self.lut = nn.Embedding(vocab, d_model)  # 定义嵌入层
        self.d_model = d_model  # 定义嵌入层维度

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)  # 嵌入并缩放

In [187]:
class PositionalEncoding(nn.Module):
    """位置编码"""

    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)  # 定义随机失活
        pe = torch.zeros(max_len, d_model)  # 初始化位置编码张量
        position = torch.arange(0, max_len).unsqueeze(1)  # 位置序号张量
        # 1/10000^(2i/d_model) = e^(-ln(10000) * 2i/d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))  # 频率
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶数位置的位置编码
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇数位置的位置编码
        pe = pe.unsqueeze(0)  # 在第0维增加一个维度
        self.register_buffer(
            "pe", pe
        )  # 将位置编码张量pe注册为buffer，使其称为模型的一部分，模型保存时缓冲区中内容也会一起被保存起来

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].detach()  # 将位置编码张量与输入张量相加
        return self.dropout(x)

In [188]:
def attention(query, key, value, mask=None, dropout=None):
    """计算缩放点积注意力"""
    d_k = query.size(-1)  # 维度
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)  # 注意力分数
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)  # 替换掩码为0的元素为-1e9，后续经softmax计算后该值会趋近于0
    p_attn = F.softmax(scores, dim=-1)  # 注意力权重
    if dropout is not None:
        p_attn = dropout(p_attn)  # 随机失活
    return torch.matmul(p_attn, value), p_attn

In [189]:
def clones(module, N):
    """克隆N个相同的模块"""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class MultiHeadedAttention(nn.Module):
    """多头注意力"""

    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0  # 维度必须能被h整除
        # 假设d_v等于d_k
        self.d_k = d_model // h  # key的维度
        self.h = h  # 注意力头的数量
        self.linears = clones(nn.Linear(d_model, d_model), 4)  # 创建4个线性层，前三个用于计算注意力，最后一个用于输出
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # 所有注意力头使用相同的掩码
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)  # 批数量
        # 先对输入进行线性变换，再将query,key,value的维度从[nbatches, seq_len, d_model]转换为[nbatches, h, seq_len, d_k]
        query, key, value = [
            l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linears, (query, key, value))
        ]
        # 批量计算注意力
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        # 转换维度[nbatches, h, seq_len, d_k] -> [nbatches, seq_len, h, d_k]，再使内存连续，再合并多个注意力头的结果
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        # 使用最后一个线性层进行线性变换
        return self.linears[-1](x)

In [193]:
"""测试多头注意力"""

d_model = 512  # 词向量维度
vocab = 10000  # 词表大小
embeddings = Embeddings(d_model, vocab)  # 词嵌入
positional_encoding = PositionalEncoding(d_model, 0.1)  # 位置编码
x = torch.tensor([[1, 2, 3, 1], [6, 5, 89, 0]])  # 输入
mask = x != 0  # 掩码
mask.unsqueeze_(-2)  # 升维
x = embeddings(x)  # 词嵌入
x = positional_encoding(x)  # 位置编码
h = 8  # 注意力头个数
mha = MultiHeadedAttention(h, d_model, dropout=0.1)
x = mha(x, x, x, mask)
print(x)
print(mha.attn)

tensor([[[ 2.1997e-01,  1.3060e+01, -7.2158e+00,  ...,  1.7397e+01,
           3.4319e+00, -1.5005e+01],
         [-6.2329e+00, -2.1985e+00,  6.4613e+00,  ..., -5.4882e+00,
           1.1677e+01, -1.3383e+00],
         [ 2.2081e+00, -6.7845e+00, -1.1202e+01,  ..., -2.5567e+00,
           1.6518e+01,  1.7285e-02],
         [-1.0606e+00,  1.4312e+01, -8.2333e+00,  ...,  2.8254e+01,
           5.8257e+00, -1.6829e+01]],

        [[-1.5205e+01,  1.7410e+01, -8.6704e+00,  ...,  3.0690e+00,
           2.4596e+01,  7.6393e+00],
         [ 7.1143e+00, -2.5405e+00,  1.1196e+00,  ..., -6.4654e+00,
           4.1107e+00, -7.2060e+00],
         [-1.3390e+01,  7.4375e+00, -3.3463e+00,  ..., -4.7618e+00,
          -1.3738e+01, -1.1500e+01],
         [-7.1136e+00,  1.5952e-01, -1.8001e+00,  ..., -1.0996e+00,
           2.1199e+01,  1.0706e+01]]], grad_fn=<ViewBackward0>)
tensor([[[[1.1111e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [

In [191]:
def subsequent_mask(size):
    """创建掩码张量"""
    attn_shape = (1, size, size)  # 注意力张量形状
    subsequent_mask = np.tril(np.ones(attn_shape), k=0)  # 创建下三角矩阵
    return torch.from_numpy(subsequent_mask)